In [ ]:
import sys
import os
sys.path.append(os.path.abspath(".."))

import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models, optimizers

from scripts.dataPreprocessingPipeline import load_images_for_resnet
from scripts.showImage import show_image, show_images

In [ ]:
images, labels, labels_list = load_images_for_resnet("../data/labeled")
show_images(images[:5])

In [ ]:
model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

model.trainable = False

In [ ]:
NUM_CLASSES = len(labels_list)

new_model = models.Sequential([
    model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(NUM_CLASSES, activation='softmax')
])

In [ ]:
new_model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = new_model.fit(
    images, labels,
    batch_size=32,
    epochs=10,
    validation_split=0.3,
    shuffle=True
)